In [33]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
# 그래프에서 한글 깨짐 방지
import platform
if platform.system() == "Darwin":          # MAC인 경우
    plt.rc('font', family ="AppleGothic")
else:
    plt.rc('font',family='Malgun Gothic')

#멧플로립에서 음수를 표시
plt.rc('axes',unicode_minus=False)

#워닝 메시지
import warnings
warnings.filterwarnings('ignore')

In [34]:
df = pd.read_csv('../data/t-1_raw_상장.csv')

In [35]:
df['FCF'] = df['OCF(천원)'] + df['ICF(천원)']
df['매출액대비잉여현금흐름'] = df['FCF'] / df['매출액(천원)']
df['총자산대비영업현금흐름'] = df['OCF(천원)'] / df['자산총계(천원)']

---
- 결측치, 무한대값 삭제

In [36]:
df = df[['회사명', '거래소코드', '부채비율', '총자본회전률', '매출액대비잉여현금흐름', 'PBR', '총자산대비영업현금흐름', '자기자본증가율', '총자본투자효율', '총자본순이익률', '매출액영업이익률', 't-1감사의견코드']]

In [37]:
df.isna().sum()
df.dropna(axis=0,inplace=True)

In [38]:
df.isin([np.inf,-np.inf]).sum()
df = df.replace([np.inf, -np.inf], np.nan)  # inf 값을 NaN으로 대체
df = df.dropna()  # NaN이 있는 행 삭제

---
- 이상치 제거 (1%)

In [39]:
# std, Mean·Median
df.describe()

,거래소코드,부채비율,총자본회전률,매출액대비잉여현금흐름,PBR,총자산대비영업현금흐름,자기자본증가율,총자본투자효율,총자본순이익률,매출액영업이익률,t-1감사의견코드
count,14716.000000,14716.000000,14716.000000,14716.000000,14716.000000,14716.000000,14716.000000,14716.000000,14716.000000,14716.000000,14716.000000
mean,78234.785947,137.238289,0.922312,-1.958144,-21.832149,0.040765,16.380499,18.061905,1.205548,-90.270031,0.005096
std,81340.164128,964.774898,0.565298,166.002738,468.534317,0.102508,187.670567,17.901886,14.051543,4501.911671,0.071210
min,20.000000,0.000000,0.000000,-19895.239494,-56563.287051,-2.187211,-807.100000,-348.610000,-318.880000,-373364.740000,0.000000
25%,15710.000000,36.880000,0.560000,-0.083781,-18.997475,-0.001988,0.000000,10.690000,-1.200000,0.600000,0.000000
50%,52860.000000,78.685000,0.830000,-0.002317,-9.394634,0.045075,4.430000,17.440000,2.680000,4.270000,0.000000
75%,101400.000000,145.585000,1.160000,0.052914,-3.768958,0.091053,13.860000,25.260000,6.590000,9.010000,0.000000
max,439090.000000,84510.240000,8.510000,411.793790,37.980402,0.782707,14501.350000,233.670000,252.030000,88.370000,1.000000


In [40]:
import numpy as np

# 컬럼 이름
column_name = '매출액영업이익률'

# 하위 1%, 상위 1% 값을 계산
lower_bound = df[column_name].quantile(0.005)
upper_bound = df[column_name].quantile(0.995)

# 1% 미만 및 1% 초과 데이터 선택 후 삭제
df = df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]


In [41]:
# import numpy as np

# # 컬럼 이름
# column_name = '부채비율'

# # 하위 1%, 상위 1% 값을 계산
# lower_bound = df[column_name].quantile(0.005)
# upper_bound = df[column_name].quantile(0.995)

# # 1% 미만 및 1% 초과 데이터 선택 후 삭제
# df = df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]

In [42]:
# import numpy as np

# # 컬럼 이름
# column_name = 'PBR'

# # 하위 1%, 상위 1% 값을 계산
# lower_bound = df[column_name].quantile(0.01)
# upper_bound = df[column_name].quantile(0.99)

# # 1% 미만 및 1% 초과 데이터 선택 후 삭제
# df = df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]

In [43]:
# import numpy as np

# # 컬럼 이름
# column_name = '자기자본증가율'

# # 하위 1%, 상위 1% 값을 계산
# lower_bound = df[column_name].quantile(0.01)
# upper_bound = df[column_name].quantile(0.99)

# # 1% 미만 및 1% 초과 데이터 선택 후 삭제
# df = df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]

---
- DP_score

In [44]:
# Scaling
scaler=MinMaxScaler()
df.iloc[:,2:-1]=scaler.fit_transform(df.iloc[:,2:-1])

In [45]:
# dp-score 계수
coefficients = {
    '부채비율' : np.exp(-0.30259864),
    '총자본회전률' : np.exp(1.0667429),
    '매출액대비잉여현금흐름' : np.exp(0.03715004),
    'PBR' : np.exp(-0.33048908),
    '총자산대비영업현금흐름' : np.exp(-0.12926251),
    '자기자본증가율' : np.exp(-0.36731349),
    '총자본투자효율' : np.exp(-0.33997948),
    '총자본순이익률' : np.exp(0.94653419),
    '매출액영업이익률' : np.exp(-0.35456208),
    'constant' : 0.71344978
}

# O-Score 계산
df['dp_score'] = coefficients['constant']
df['dp_score'] += coefficients['부채비율'] * df['부채비율']
df['dp_score'] += coefficients['총자본회전률'] * df['총자본회전률']
df['dp_score'] += coefficients['매출액대비잉여현금흐름'] * df['매출액대비잉여현금흐름']
df['dp_score'] += coefficients['PBR'] * df['PBR']
df['dp_score'] += coefficients['총자산대비영업현금흐름'] * df['총자산대비영업현금흐름']
df['dp_score'] += coefficients['자기자본증가율'] * df['자기자본증가율']
df['dp_score'] += coefficients['총자본투자효율'] * df['총자본투자효율']
df['dp_score'] += coefficients['총자본순이익률'] * df['총자본순이익률']
df['dp_score'] += coefficients['매출액영업이익률'] * df['매출액영업이익률']

In [46]:
df.shape

(14568, 13)

In [47]:
df[df['t-1감사의견코드']==1]['dp_score'].mean()

5.270520170808479

- 2022년 신용평가
    - AA : 5.371351
    - A : 5.478527
    - BBB : 5.469861
    - BB : 5.390075
    - B : 5.204838
    - C : 4.793609


In [48]:
df['dp_score'].describe()

count    14568.000000
mean         5.539700
std          0.261150
min          3.367792
25%          5.405715
50%          5.530197
75%          5.667366
max          7.817149
Name: dp_score, dtype: float64